In [ ]:
from tabulate import tabulate
from pydantic import BaseModel
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.animation import FuncAnimation, PillowWriter

EXAMPLE = "../example.txt"
EXAMPLE_TREE = "../example_tree.txt"
INPUT = "../input.txt"

In [ ]:
EXAMPLE_H=7
EXAMPLE_W=11
INPUT_H=103
INPUT_W=101

In [ ]:
class Coords(BaseModel):
    x: int
    y: int

In [ ]:
class Robot(BaseModel):
    position: Coords
    velocity: Coords
    quadrant: Coords = Coords(x=-1, y=-1)

In [ ]:
def get_robots(input_file_name):
    robots = []
    with open(input_file_name, 'r') as f:
        for line in f:
            # p=0,4 v=3,-3
            p, v = line.strip().replace("\n", "").split(" ")
            _, p = p.split('=')
            px, py = p.split(',')
            position = Coords(x=int(px), y=int(py))
            _, v = v.split('=')
            vx, vy = v.split(',')
            velocity = Coords(x=int(vx), y=int(vy))
            robot = Robot(position=position, velocity=velocity)
            robots.append(robot)
    return robots

In [ ]:
robots = get_robots(EXAMPLE)
print(tabulate(robots))

In [ ]:
def move_robot(robot: Robot, seconds, height, width):
    robot.position.x += (robot.velocity.x * seconds)
    robot.position.x = robot.position.x % width
    robot.position.y += (robot.velocity.y * seconds) % height
    robot.position.y = robot.position.y % height


In [ ]:
print(robots[-2])
for s in range(5):
    move_robot(robots[-2], 1, EXAMPLE_H, EXAMPLE_W)
    print(robots[-2])

In [ ]:
def find_quadrant(robot, height, width):
    quadrant_x, quadrant_y = -1, -1
    if robot.position.x < width // 2:
        quadrant_x = 0
    elif robot.position.x > width // 2:
        quadrant_x = 1
    if robot.position.y < height // 2:
        quadrant_y = 0
    elif robot.position.y > height // 2:
        quadrant_y = 1
    robot.quadrant = Coords(x=quadrant_x, y=quadrant_y)

In [ ]:
def part_1(input_file_name, height, width):
    robots = get_robots(input_file_name)
    robots_per_quadrant = {}
    for i in [0, 1]:
        for j in [0, 1]:
            robots_per_quadrant[(i, j)] = 0
    for robot in robots:
        move_robot(robot, 100, height, width)
        find_quadrant(robot, height, width)
        if robot.quadrant.x != -1 and robot.quadrant.y != -1:
            robots_per_quadrant[(robot.quadrant.x, robot.quadrant.y)] += 1
    result = m.prod(robots_per_quadrant.values())
    print(result)
        

In [ ]:
part_1(EXAMPLE, EXAMPLE_H, EXAMPLE_W)

In [ ]:
part_1(INPUT, INPUT_H, INPUT_W)

For Part 2, best way I found is to 
- generate a gif of the robot positions
- notice that every 100 frames or so, they seem to congregate
- generate a gif with these specific frames
- notice a goddamn christmas tree in one frame

In [ ]:
def get_grid(robots, height, width):
    grid = [[0 for _ in range(width)] for _ in range(height)]
    for robot in robots:
        grid[robot.position.y][robot.position.x] = 1
    return np.array(grid)

In [ ]:
def generate_gif(input_file_name, height, width, frames=10):
    fig, ax = plt.subplots()

    def update(frame, robots, height, width):
        ax.clear()
        grid = get_grid(robots, height, width)
        # Move the robots 101 times, to get the next special frame
        for robot in robots:
            move_robot(robot, 101, height, width)
        im = ax.imshow(grid, cmap='viridis', interpolation='nearest')
        ax.set_title(f"Frames: {frame}")
        return im,

    robots = get_robots(input_file_name)
    # Move the robots 8 times, to get the initial special frame
    for _ in range(8):
        for robot in robots:
            move_robot(robot, 1, height, width)
    ani = FuncAnimation(fig, update, frames=frames, fargs=(robots, height, width), interval=100, repeat=False)
    ani.save("robot_dance.gif", writer=PillowWriter(fps=10))

In [ ]:
def part_2():
    generate_gif(INPUT, INPUT_H, INPUT_W, frames = 70)

In [ ]:
part_2()